In [3]:
#!/usr/bin/env python3
import numpy as np
import transformations as tf

/home/ajinkya/Software/ajinkya/active_champ/plots/transformations.py:1928: UserWarning: No module named '_transformations'
  warnings.warn(str(err))


In [4]:
def find_axis_in_camera_frame(pt, ref):
    quat = ref[3:]
    t = tf.quaternion_matrix(quat)
    pt = np.append(pt, 1.)
    new_pt = t.dot(pt)[:3]
    return new_pt

In [5]:
def find_axis_in_global_frame(pt):
    quat = np.array([ 0., -0.707, 0., 0.707])
    t = tf.quaternion_matrix(quat)
    pt = np.append(pt, 1.)
    new_pt = t.dot(pt)[:3]
    return new_pt

In [6]:
def find_axis_in_global_frame_from_quat(quat_in):
    t1 = tf.quaternion_matrix(quat_in)
    x = np.array([0., 0., 1. ,1.])
    pt = t1.dot(x)[:3]
    new_pt = find_axis_in_global_frame(pt)
    return new_pt

In [7]:
def calculate_error(vec, ax_mat):
    abs_err = []
    for ax in ax_mat:
        abs_err.append(1. - abs((np.dot(ax.T, vec))/(np.linalg.norm(ax)* np.linalg.norm(vec))))
    return np.array(abs_err)

In [8]:
## Axis are wrt to some origin. ZWe need to convert thtm wrt to a common frame for comparison. 
## We can convert them to a transform wrt Camera frame using the rigid body parms we know

champ_center = np.reshape([-0.0579273883048,
                            -0.311441705219,
                            -0.0208280461233,
                            -0.0499914065927,
                            -0.298867207351,
                            -0.0238727412394,
                            -0.0984513610544,
                            -0.225156339526,
                            -0.0491073035833,
                            -0.101186334794,
                            -0.256404057374,
                            -0.0406491611428,
                            -0.119080652013,
                            -0.304403051426,
                            -0.0232807446221,
                            -0.0754330371917,
                            -0.374393782435,
                            -0.00848623662944,
                            -0.0871876785983,
                            -0.274225518406,
                            -0.034927378059,
                            -0.006562,
                            -0.301980,
                            -0.023111,
                            -0.0286787201994,
                            -0.290330267038,
                            -0.0314991565349,
                            -0.0760493750511,
                            -0.306312812404,
                            -0.0251133626187,], (10,3))


champ_axes = np.reshape([0.722673934532,
                        0.119318495351,
                        0.680442337438,
                        0.0224434052029,
                        0.0235159079144,
                        0.694010247522,
                        0.113583548939,
                        0.710560029709,
                        0.192761595646,
                        0.718363130633,
                        0.00669569593423,
                        0.668395502264,
                        0.663661905128,
                        0.0676870933332,
                        0.736266248102,
                        0.113504823615,
                        0.214279293235,
                        0.711759909403,
                        0.0208679089192,
                        0.668615544416,
                        0.719987743792,
                        0.132129474142,
                        0.680902202812,
                        0.0230573428179,
                        0.704438577349,
                        0.21875104304,
                        0.672519041632,
                        0.0602694827783,
                        0.704569,
                        0.041503,
                        0.705545,
                        0.063760,
                        0.0160993054181,
                        0.694245272608,
                        0.0731925861839,
                        0.715826207368,
                        0.673281201678,
                        0.0744726656792,
                        0.706995684197,
                        0.203232251525,], (10,4))

dummy = np.zeros((champ_axes.shape[0], champ_axes.shape[1]))
dummy[:,1:] = champ_axes[:, :-1]
dummy[:,0] = champ_axes[:, -1]
champ_axes = dummy 

champ_radius = np.array([0.458099021896,
                        0.442272342186,
                        0.380014085967,
                        0.410851315051,
                        0.463415352235,
                        0.521087147813,
                        0.424585162326,
                        0.441978,
                        0.431572381497,
                        0.454284369763,])

champ_center_global = []
for pt in champ_center:
    pt_in_global =  find_axis_in_global_frame(pt)
    champ_center_global.append(pt_in_global)

champ_axes_global = []
for axis in champ_axes:
    axis_in_global =  find_axis_in_global_frame_from_quat((axis))
    champ_axes_global.append(axis_in_global)

In [9]:
gt_axis = np.array([0., 0., 1.])
gt_rad = 0.45
gt_center = np.array([-0.07834291, -0.3022653 , -0.02670656])

abs_err_center = []
for p in champ_center:
    abs_err_center.append(np.linalg.norm(gt_center - p))
print("\nCHAMP Absolute Error in Center:")
mean_abs_err_center = np.mean(abs_err_center, axis=0)
print("Mean: ", mean_abs_err_center)
std_abs_err_center = np.std(abs_err_center, axis=0)
print("std: ", std_abs_err_center)

abs_err = calculate_error(gt_axis, champ_axes_global)
print("\nCHAMP Absolute Error in Axis:")
mean_abs_err = np.mean(abs_err, axis=0)
print("Mean: ", mean_abs_err)
std_abs_err = np.std(abs_err, axis=0)
print("std: ", std_abs_err)

abs_err_rad = abs(abs(champ_radius) - gt_rad)
print("\nCHAMP Absolute Error in Radius:")
mean_abs_err_rad = np.mean(abs_err_rad, axis=0)
print("Mean: ", mean_abs_err_rad)
std_abs_err_rad = np.std(abs_err_rad, axis=0)
print("std: ", std_abs_err_rad)


CHAMP Absolute Error in Center:
Mean:  0.04617211149519965
std:  0.02384160580124977

CHAMP Absolute Error in Axis:
Mean:  0.016480877876773613
std:  0.012822659729494035

CHAMP Absolute Error in Radius:
Mean:  0.026561260468000013
std:  0.02408839271614798


In [28]:
abs_err_center

0.16432547115731433

In [10]:
## Act-CHAMP
actCHAMP_center = np.reshape([-0.043987104936,
-0.384950073276,
0.00637353341802,
-0.0837461418502,
-0.297670329997,
-0.0253595877004,
-0.0723670079919,
-0.273478530768,
-0.0354516102802,
-0.0979015384476,
-0.406774924811,
0.00180562693553,
-0.013317255625,
-0.352990112405,
-0.0126896848087,
-0.0936337220879,
-0.277541346346,
-0.0426369480588,
-0.102467230575,
-0.26236496224,
-0.0412949997243,
-0.126543795813,
-0.236732886214,
-0.0476438265906,
-0.059589343518,
-0.250017941012,
-0.0420165737243,
-0.172757135358,
-0.359270128497,
-0.0143419314244,], (10,3))


actCHAMP_axes = np.reshape([-0.15857694725,
-0.703320792394,
0.0915568134185,
0.68688468079,
0.673058808127,
0.0665408670808,
0.705832973444,
0.21062755615,
0.707651016436,
-0.191009473411,
-0.67789777211,
0.0565688127115,
-0.183501896761,
-0.706509833286,
0.0508829733671,
0.681602400507,
-0.107909582158,
-0.709184326103,
0.080392768413,
0.692062220089,
0.674881361873,
-0.0620850967931,
0.728399810505,
0.100569897134,
0.67078246909,
0.0436806412628,
0.702732437901,
0.233045063157,
0.0267141125633,
0.664175692653,
-0.215142709663,
0.715451340035,
0.677918631704,
-0.0372074484826,
0.730093452213,
0.077495068244,
0.663555797151,
-0.0474787384675,
0.721666966439,
0.191406016133,], (10,4))

dummy = np.zeros((actCHAMP_axes.shape[0], actCHAMP_axes.shape[1]))
dummy[:,1:] = actCHAMP_axes[:, :-1]
dummy[:,0] = actCHAMP_axes[:, -1]
actCHAMP_axes = dummy 

actCHAMP_radius = np.array([0.532896139343,
0.448360117058,
0.420608354834,
0.561373593083,
0.493565529174,
0.429025079363,
0.41584310955,
0.399219057777,
0.395476201613,
0.533190324624,])

actCHAMP_axes_global = []
for axis in actCHAMP_axes:
    actCHAMP_axis =  find_axis_in_global_frame_from_quat(axis)
    actCHAMP_axes_global.append(actCHAMP_axis)

actCHAMP_abs_err = calculate_error(gt_axis, actCHAMP_axes_global)
print("\nActCHAMP Absolute Error in Axis:")
actCHAMP_mean_abs_err = np.mean(actCHAMP_abs_err, axis=0)
print("Mean: ", actCHAMP_mean_abs_err)
actCHAMP_std_abs_err = np.std(actCHAMP_abs_err, axis=0)
print("std: ", actCHAMP_std_abs_err)

actCHAMP_abs_err_rad = actCHAMP_radius - gt_rad
print("\nActCHAMP Absolute Error in Radius:")
actCHAMP_mean_abs_err_rad = np.mean(actCHAMP_abs_err_rad, axis=0)
print("Mean: ", actCHAMP_mean_abs_err_rad)
actCHAMP_std_abs_err_rad = np.std(actCHAMP_abs_err_rad, axis=0)
print("std: ", actCHAMP_std_abs_err_rad)


actCHAMP_abs_err_center = []
for p in actCHAMP_center:
    actCHAMP_abs_err_center.append(np.linalg.norm( p - gt_center))

print("\nActCHAMP Absolute Error in center:")
actCHAMP_mean_abs_err_center = np.mean(actCHAMP_abs_err_center, axis=0)
print("Mean: ", actCHAMP_mean_abs_err_center)
actCHAMP_std_abs_err_center = np.std(actCHAMP_abs_err_center, axis=0)
print("std: ", actCHAMP_std_abs_err_center)



ActCHAMP Absolute Error in Axis:
Mean:  0.02684611952493652
std:  0.0196108223351446

ActCHAMP Absolute Error in Radius:
Mean:  0.012955750641899983
std:  0.05868616959443605

ActCHAMP Absolute Error in center:
Mean:  0.06616491469403216
std:  0.034082591175127214


In [35]:
np.shape(champ_axes[:,:3])

(10, 3)

In [15]:
## CHANGEPOINT Analysis
champ_cp = np.reshape([-0.00193311273984,
0.142402253709,
-0.00848558509461], (1, 3))

ActCHAMP_cp = np.reshape([-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.000867558914176,
0.141855125978,
-0.0049918187279,
-0.00193311273984,
0.142402253709,
-0.00848558509461,
-0.00193311273984,
0.142402253709,
-0.00848558509461,], (6, 3))

# gt_cp = np.mean(np.concatenate((champ_cp, ActCHAMP_cp), axis=0), axis=0)
# print("gt_cp: ", gt_cp)

gt_cp = np.array([-0.00172, 0.14229282, -0.00778685])

print("Error in CHAMP cp")
champ_cp_err = []
for p in champ_cp:
    champ_cp_err.append(np.linalg.norm(p - gt_cp))

mean_champ_cp_err = np.mean(champ_cp_err)
print("Mean err in CHAMP cp: ", mean_champ_cp_err)
std_champ_cp_err = np.std(champ_cp_err)
print("std in err in CHAMP cp: ", std_champ_cp_err)

print("\n\nError in ActCHAMP cp")
ActCHAMP_cp_err = []
for p in ActCHAMP_cp:
    ActCHAMP_cp_err.append(np.linalg.norm(p - gt_cp))

mean_ActCHAMP_cp_err = np.mean(ActCHAMP_cp_err)
print("Mean err in ActCHAMP cp: ", mean_ActCHAMP_cp_err)
std_ActCHAMP_cp_err = np.std(ActCHAMP_cp_err)
print("std in err in ActCHAMP cp: ", std_ActCHAMP_cp_err)

Error in CHAMP cp
Mean err in CHAMP cp:  0.000738663325871306
std in err in CHAMP cp:  0.0


Error in ActCHAMP cp
Mean err in ActCHAMP cp:  0.001108007836450069
std in err in ActCHAMP cp:  0.0008258794327705039
